In [1]:
#Imports
#standard ds libraries
import pandas as pd
import numpy as np

# my acquire and prepare file
import acquire
import prepare

#env file to import datasets
import env

# import splitting functions
from sklearn.model_selection import train_test_split

### 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
DB_URL = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

In [3]:
attendance = pd.read_sql('SELECT * FROM attendance', DB_URL)
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
#renaming student columna
attendance.rename(columns = {'Unnamed: 0':'student'}, inplace = True)

In [5]:
#melting down
attendance = attendance.melt(id_vars = 'student',
                  var_name = 'date',
                  value_name = 'attendance')

attendance

,student,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [6]:
#Rename attendance variables
attendance['attendance'] = attendance['attendance'].replace({'P':1, 'A':0, 'T':0.9, 'H':0.5})
attendance

,student,date,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9
5,Jane,2018-01-02,1.0
6,Billy,2018-01-02,0.9
7,John,2018-01-02,0.9
8,Sally,2018-01-03,0.9
9,Jane,2018-01-03,0.9


In [7]:
attendance.groupby('student').mean()

,attendance
student,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


### 2. Coffee Levels
Read the coffee_levels table.
Transform the data so that each carafe is in it's own column.
Is this the best shape for the data?

In [8]:
#a. read the coffee level tables
coffee_levels = pd.read_sql('SELECT * FROM coffee_levels', DB_URL)
coffee_levels

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [9]:
#b. Transform the data so that each carafe is in it's own column.
coffee_levels = coffee_levels.pivot(index='hour', columns='coffee_carafe', values= 'coffee_amount')
coffee_levels

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [11]:
#renaming columns
coffee_levels.columns = ['carafe_x', 'carafe_y', 'carafe_z']
coffee_levels

,carafe_x,carafe_y,carafe_z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [12]:
#c is this the best shape for the data?
#Yes, because now you can view the carafes data more clearly as it relates to hour.

### 3. Cake recipes

In [13]:
# a. Read the cake_recipes table. This data set contains cake tastiness scores 
# for combinations of different recipes, oven rack positions, and oven temperatures.
cake_recipes = pd.read_sql('SELECT * FROM cake_recipes', DB_URL)
cake_recipes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [14]:
#b. Tidy the data as necessary.
cake_melt= cake_recipes.melt(id_vars = 'recipe:position',
                  var_name = 'temperature')

In [15]:
cake_melt

,recipe:position,temperature,value
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [16]:
#renaming columns
cake_melt.rename(columns = {'recipe:position':'recipe_position'}, inplace = True)
cake_melt

,recipe_position,temperature,value
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [17]:
#Splitting recipe and position
recipe_position_df = cake_melt.recipe_position.str.split(':', expand=True)
recipe_position_df.columns = ['recipe', 'rack position']
recipe_position_df.head()

,recipe,rack position
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom


In [18]:
#adding back to df
cake_recipes_2 = pd.concat([cake_melt, recipe_position_df], axis=1).drop(columns='recipe_position')
cake_recipes_2.head()

,temperature,value,recipe,rack position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom


In [19]:
# c. Which recipe, on average, is the best? recipe b
cake_recipes_2.groupby('recipe').mean()

,value
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


In [20]:
#d. Which oven temperature, on average, produces the best results? 
#275
cake_recipes_2.groupby('temperature').mean()

,value
temperature,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [21]:
# e. Which combination of recipe, rack position, and temperature gives the best result? 
# recipe b, bottom rack, 300 degrees
cake_recipes_2.sort_values(by = 'value', ascending = False)

,temperature,value,recipe,rack position
26,300,99.248541,b,bottom
24,300,98.786784,a,bottom
19,275,98.594881,b,top
7,225,96.873178,d,top
4,225,96.470207,c,bottom
11,250,95.224151,b,top
20,275,92.893227,c,bottom
21,275,92.098049,c,top
13,250,82.795477,c,top
3,225,82.455004,b,top
